In [98]:
# Install a pip package in the current Jupyter kernel
import sys
!{sys.executable} -m pip install plotly


[notice] A new release of pip available: 22.2 -> 24.0
[notice] To update, run: C:\Python39\python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.2 -> 24.0
[notice] To update, run: C:\Python39\python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable
     ---------------------------------------- 15.7/15.7 MB 8.1 MB/s eta 0:00:00
  Using cached tenacity-8.2.3-py3-none-any.whl (24 kB)

[notice] A new release of pip available: 22.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import sqlite3
import os
import pandas as pd
from shapely.geometry import LineString
from shapely.geometry.polygon import Polygon
from shapely.geometry import Point
import geopandas as gpd
from shapely import wkt
import folium

from shapely.geometry import MultiPolygon, Polygon
from shapely.wkt import loads
import re

import numpy as np
import matplotlib.pyplot as plt


spatialite_path = "C:/Users/user/Documents/mod_spatialite-5.1.0-win-amd64"
os.environ["PATH"] = spatialite_path + ";" + os.environ["PATH"]

# create a new sqlite data base
conn = sqlite3.connect("BPN_1.sqlite")
conn.enable_load_extension(True)
conn.load_extension("mod_spatialite")

# mod_spatialite (recommended)
conn.execute('SELECT load_extension("mod_spatialite")')

# Create a cursor object to execute SQL queries
cursor = conn.cursor()

# initiate a spatial metadata for the sqlite database 
cursor.execute("SELECT InitSpatialMetadata(1);")



In [2]:
# Checking spatialite version 
res = cursor.execute("SELECT spatialite_version()")
print(res.fetchone())

# Checking Sqlite Version
res1 = cursor.execute("SELECT sqlite_version()")
print(res1.fetchone())

('5.1.0',)
('3.45.1',)


In [82]:
# import pandas module  
import pandas as pd 
import ast


def csv2coord(file):
    result = []
    # making dataframe  
    df = pd.read_csv(file)  
    vertices = df.vertices_str 
    
    for x in vertices:
        temp = ast.literal_eval(x)
        templist = []
        for y in temp:
            templist.append([y[0], y[1]])
        result.append(templist)
    
    return result

# Example = csv2coord("dataRekap.csv")
# Example = csv2coord("testtopo.csv")

# print(Example)

In [83]:
import matplotlib.pyplot as plt
import re

def customQuery(Query):
    database = sqlite3.connect('BPN_1.sqlite')
    dataframe = pd.read_sql_query(Query, database)
#     print(dataframe)
    return dataframe
    
# second = customQuery("select ST_Intersects(r1.geom, r2.geom) as check1 from spatial_data r1 cross join spatial_data r2 where r1.id = 4 and r2.id = 2;")

def customFetch(Fetch):
    Fetch = Fetch
    cursor.execute(Fetch)
    res = cursor.fetchone()
    return res[0]

def customFetchAll(Fetch):
#     cursor.execute("select ST_Touches(r1.geom, r2.geom) as check1 from spatial_data r1 cross join spatial_data r2 where r1.id = 4 and r2.id = 2;")
    cursor.execute(Fetch)
    res = cursor.fetchall()
#     print(res)
    return res


def plotpolygon(data, figsize=(10, 10), point_size=100):
    plt.figure(figsize=figsize)
    
    # Initialize plot limits
    x_min, x_max, y_min, y_max = float('inf'), float('-inf'), float('inf'), float('-inf')

    for item in data:
        # Extract the polygon points from the item
        polygon_string = item[2]
        # Use regular expressions to find all numbers in the string
        numbers = re.findall(r'[-+]?\d*\.\d+|\d+', polygon_string)
        # Convert the string numbers to floats and pair them as points
        points = [(float(numbers[i]), float(numbers[i + 1])) for i in range(0, len(numbers), 2)]
        
        # Separate the x and y coordinates for plotting
        x = [point[0] for point in points]
        y = [point[1] for point in points]
        
        # Update plot limits
        x_min, x_max = min(x_min, min(x)), max(x_max, max(x))
        y_min, y_max = min(y_min, min(y)), max(y_max, max(y))

        # Plot the polygon
        plt.fill(x, y, alpha=0)  # Fill the polygon with half transparency
        plt.plot(x, y, 'r-')  # Outline the polygon in red
        
        # Plot the points with increased size
        plt.scatter(x, y, color='yellow', edgecolor='black', s=point_size, zorder=5)
        
        # Annotate points with their coordinates
#         for i, txt in enumerate(points):
#             plt.annotate(f'({txt[0]}, {txt[1]})', (txt[0], txt[1]), textcoords="offset points", xytext=(0,10), ha='center')

    plt.xlim(x_min - 1, x_max + 1)
    plt.ylim(y_min - 1, y_max + 1)
    plt.xlabel('X')
    plt.ylabel('Y')
    plt.title('Multiple Polygons Plot with Points')
    plt.grid(True)
    plt.show()

# Example usage with multiple polygon data
data = [
    (1, 4.0, 'POLYGON((10 10, 12 10, 12 12, 10 12, 10 10))'),
    (2, 4.0, 'POLYGON((13 10, 15 10, 15 12, 13 12, 13 10))')  # Example of another polygon
]

# plotpolygon(data, figsize=(10, 10), point_size=200)


In [84]:
import plotly.graph_objects as go
import re

# Your data
data2 = [
    [[170046.656369714, 538690.0459865994], [170066.58773438, 538688.1754761626], [170070.8320303895, 538723.1804464505], [170050.4496688225, 538724.8305879902], [170046.656369714, 538690.0459865994]],
    [[171100.9989847714, 538247.1770387553], [171100.7881967803, 538254.0738183243]],
    [[171199.819632471, 538405.9068421484], [171224.6684580127, 538406.148439336]],
    [[171224.6684580127, 538406.148439336], [171224.2794046072, 538420.8933075093], [171199.5805551606, 538420.6549044631], [171199.819632471, 538405.9068421484]],
    [[171199.819632471, 538405.9068421484], [171200.0628073911, 538391.2588605142], [171225.061608282, 538391.5037156149], [171224.6684580127, 538406.148439336]],
    [[170183.1671263177, 538714.0709656392], [170204.063814195, 538712.3992879571], [170201.7357373573, 538683.79386787], [170198.3503241029, 538649.9125828154], [170186.160369056, 538651.2286435228], [170182.4075368875, 538652.1321094693], [170176.1497665878, 538656.6250519279], [170180.7916553698, 538691.3877633462], [170181.8964477109, 538700.7010195751], [170183.1671263177, 538714.0709656392], [170204.063814195, 538712.3992879571]],
    [[170293.5604201219, 538643.7706410174], [170309.1592909582, 538643.5829480983], [170309.201128984, 538655.6793454782], [170293.7094431325, 538655.1717339354], [170293.5604201219, 538643.7706410174]]
]

def plotBefore(data, point_size=5):
    fig = go.Figure()

    for polygon in data:
        x, y = zip(*polygon)
        fig.add_trace(go.Scatter(x=x, y=y, fill='toself', mode='lines+markers', line_color='red', marker=dict(size=point_size, color='yellow', line=dict(color='black', width=2))))

    # Update layout for black axis lines
    fig.update_layout(
        title='Parcels BEFORE cleaning',
        plot_bgcolor='rgba(0,0,0,0)',  # Transparent background
        xaxis_title='X',
        yaxis_title='Y',
        showlegend=False,
        xaxis=dict(showline=True, linewidth=2, linecolor='black', mirror=True),
        yaxis=dict(showline=True, linewidth=2, linecolor='black', mirror=True)
    )

    fig.show()


def plotAfter(data, point_size=5):
    fig = go.Figure()

    for item in data:
        # Extract the polygon points from the item
        polygon_string = item[2]
        # Use regular expressions to find all numbers in the string
        numbers = re.findall(r'[-+]?\d*\.\d+|\d+', polygon_string)
        # Convert the string numbers to floats and pair them as points
        points = [(float(numbers[i]), float(numbers[i + 1])) for i in range(0, len(numbers), 2)]
        x, y = zip(*points)

        # Plot the polygon
        fig.add_trace(go.Scatter(x=x, y=y, fill="toself", mode='lines+markers', line_color='red', marker=dict(size=point_size, color='yellow', line=dict(color='black', width=2))))

    fig.update_layout(
        title='Parcels AFTER cleaning',
        xaxis_title='X',
        yaxis_title='Y',
        showlegend=False,
        plot_bgcolor='rgba(0,0,0,0)',
        xaxis=dict(showline=True, linewidth=2, linecolor='black', mirror=True),
        yaxis=dict(showline=True, linewidth=2, linecolor='black', mirror=True)
    )

    fig.show()

# Example usage with multiple polygon data
data = [
    (1, 4.0, 'POLYGON((10 10, 12 10, 12 12, 10 12, 10 10))'),
    (2, 4.0, 'POLYGON((13 10, 15 10, 15 12, 13 12, 13 10))')
]



In [156]:
# this is the class for geometry
class Geometry:
###     Phase initialization 
#     constructor for geometry
    def __init__(self, tableName, Coords, SRID):
        self.Coords = Coords
        self.SRID = SRID
        self.tableName = tableName
    
    def initTable(self):
        cursor.execute(f"CREATE TABLE IF NOT EXISTS {self.tableName} (id INTEGER PRIMARY KEY, geotypes VARCHAR, geom BLOB, geom2 BLOB);")
        conn.commit()
        print(f'Table successfully created with name {self.tableName} and SRID {self.SRID}')

###     Phase creating point, line, and polygon
    def convertPoint(self):
        Point = ['Point({} {})'.format(x, y) for sublist in self.Coords for x, y in sublist]
        for x in Point:
            print(f"INSERT INTO {self.tableName} (geotypes, geom) VALUES ('Point', ST_GeomFromText('{x}',{self.SRID}));")
#             cursor.execute(f"INSERT INTO {self.tableName} (geotypes, geom) VALUES ('Point', ST_GeomFromText('{x}',{self.SRID}));")
        conn.commit()
        print(f"Point successfully inserted into {self.tableName}")
        return self

    def convertLinestring(self):
        Line = [LineString(points) for points in self.Coords]
        for x in Line:
            print(f"INSERT INTO {self.tableName} (geotypes, geom) VALUES ('Linestring', ST_GeomFromText('{x}',{self.SRID}));")
            cursor.execute(f"INSERT INTO {self.tableName} (geotypes, geom) VALUES ('Linestring', ST_GeomFromText('{x}',{self.SRID}));")
        conn.commit()
        print(f"LineString successfully inserted into {self.tableName}")
        return self

    def convertPolygon(self):
        failpolygons = [points for points in self.Coords if len(points) <= 2]
        Poly = [Polygon(points) for points in self.Coords if len(points) > 2]
        Line = [LineString(points) for points in self.Coords if len(points) > 2]
#         print(Poly)
        for x in range(0, len(Poly)):
#             print(f"INSERT INTO {self.tableName} (geotypes, geom, geom2) VALUES ('Polygon', ST_GeomFromText('{Poly[x]}',{self.SRID}), ST_GeomFromText('{Line[x]}',{self.SRID}));")
            cursor.execute(f"INSERT INTO {self.tableName} (geotypes, geom, geom2) VALUES ('Polygon', ST_GeomFromText('{Poly[x]}',{self.SRID}), ST_GeomFromText('{Line[x]}',{self.SRID}));")
#         for x in Poly:
#             print(f"INSERT INTO {self.tableName} (geotypes, geom, geom2) VALUES ('Polygon', ST_GeomFromText('{x}',{self.SRID}), ST_GeomFromText('{x}',{self.SRID}));")
#             cursor.execute(f"INSERT INTO {self.tableName} (geotypes, geom, geom2) VALUES ('Polygon', ST_GeomFromText('{x}',{self.SRID}));")
        conn.commit()
        print(f"Polygon successfully inserted into {self.tableName}")
        return self
    
###     Custom Query and Fetch        
    def customQuery(self, Query): 
        self.Query = Query
        database = sqlite3.connect('BPN_1.sqlite')
        dataframe = pd.read_sql_query(self.Query, database)
        return dataframe
    
    def customFetchOne(self, Fetch):
        self.Fetch = Fetch
        cursor.execute(self.Fetch)
        res = cursor.fetchone()
        return res[0]
    
    def customFetchAll(self, Fetch):
        self.Fetch = Fetch
        cursor.execute(self.Fetch)
        self.res = cursor.fetchall()
        return self.res


###     Phase geometry analysis 
    #     method to check duplicate and return the location and value of the duplicate data 
    def conv2coor(self, data):
        Coords = [
            [[int(coord) for coord in point.split()] 
             for point in re.findall(r'(\d+ \d+)', polygon[0])] 
            for polygon in data
        ]
        return Coords

    
    def checkDuplicate(self, Coords):
        duplicate = []
    #       to store the unique value 
        seen_elements = set()
    #     accessing the index through enumerate
        for i, sublist in enumerate(Coords):
            for j, element in enumerate(sublist):
                if tuple(element) in seen_elements and tuple(element) != tuple(sublist[len(sublist)-1]):
#                     print(f'Duplicate found at location ({i}, {j}): {element}')
                    duplicate.append(tuple(element))
                else:
                    seen_elements.add(tuple(element))
        Lduplicate= len(duplicate)
        return [duplicate, Lduplicate]

    # resDuplicate = checkDuplicate(Coords)

    def resDuplicate(self):
        singleData = self.customFetchAll(f"select id from {self.tableName}")
        totalDupl = 0
        resDuplicate = []
        for x in singleData: 
            singleResult = self.customFetchAll(f"select ST_AsText(geom) from {self.tableName} where id={x[0]}")
            singleCoords = self.conv2coor(singleResult)
            tempDuplicate = self.checkDuplicate(singleCoords)
            if tempDuplicate[1] == 1:
                totalDupl = totalDupl+1
                resDuplicate.append(tempDuplicate[0])
#         print(totalDupl)
#         print(resDuplicate)
        
        return [resDuplicate, totalDupl]
    

    def checkCrosses(self, types): 
        self.resCrosses = []
        self.result_geom = []
        self.dataCross = []
        df = self.customQuery(f"select id from {self.tableName} where geotypes='{types}'")
        col_list = df["id"].values.tolist()
        for i in range(len(col_list)):
            for j in range(i + 1, len(col_list)):
                test = self.customFetchOne(f"select ST_Overlaps(r1.geom, r2.geom) as check1 from {self.tableName} r1 cross join {self.tableName} r2 where r1.id = {col_list[i]} and r2.id = {col_list[j]};")
                if test == 1:
#                     print("intersect in polygon id")
#                     print(col_list[i], col_list[j])
                    data_intersect = tuple((col_list[i], col_list[j]))
                    self.dataCross.append(data_intersect)
                    self.resCrosses.append(col_list[i])

        for i in self.resCrosses:      
            dataFetch = self.customFetchOne(f"select asText(geom) from {self.tableName} where id={i}")
            self.result_geom.append(dataFetch)
#         print("objects that are crossed:")
#         print(self.result_geom)
#         print(self.dataCross)
        self.LcheckCrosses= len(self.dataCross)

        return [self.result_geom, self.dataCross, self.LcheckCrosses]
    

    def checkIsclosed(self):
        dataList = self.customFetchAll(f"SELECT id, ST_IsClosed(geom) as close, asText(geom) from {self.tableName} where geotypes='Linestring'")
        self.resClose = []
        self.resCpoint = []
         # Iterate over each tuple in the list
        for item in dataList:
            # Check if the second value (index 1) is less than 1
            if item[1] == 0:
                # If it is, append the entire tuple to the result list
                self.resClose.append(item)
#         print("The line is not closed, possibility for short or over degenerate")
#         print(self.resClose)

        # Iterate over each tuple in the list
        for item in self.resClose:
            # Extract the LINESTRING string
            linestring = item[2]

            # Remove the 'LINESTRING(' and ')' parts from the string
            coordinates_part = linestring.replace('LINESTRING(', '').replace(')', '')

            # Split the string by commas and spaces to get coordinates
            coordinates = coordinates_part.split(', ')

            # Take the last coordinate and split it to get x and y
            x, y = map(float, coordinates[-1].split())

            # Create a Point object
            point = Point(x, y)

            # Append the Point object to the list
            self.resCpoint.append(point)
            
        self.LcheckIsclosed= len(self.resCpoint)
#         print("The end points of each unclose linestring:")
#         print(self.resCpoint)
        return self.resCpoint, self.LcheckIsclosed
            
            
    def checkIsValid(self, geotypes):
        dataList = self.customFetchAll(f"SELECT id, ST_IsValid(geom), ST_IsValidReason(geom), ST_AsText(geom) FROM {self.tableName} where geotypes='{geotypes}';")
        resValid = []
        points = []
        self.return_data = []
        for item in dataList:
            if item[1] == 0:
                resValid.append((item))
                self.return_data.append(tuple((item[0], item[3])))
#         print("Polygon that has error")
#         print(resValid)

        # Iterate over each tuple in the list
        for item in resValid:
            # Extract the polygon string
            polygon_string = item[3]
            # Remove the 'POLYGON((' and '))' parts from the string
            coordinates_part = polygon_string.replace('POLYGON((', '').replace('))', '')
            # Split the string by commas and spaces to get coordinates
            coordinates = coordinates_part.split(', ')
            # Take the last coordinate and split it to get x and y
            x, y = map(float, coordinates[-1].split())
            # Create a Point object
            point = Point(x, y)
            # Append the Point object to the list
            points.append(point)
        
        self.LcheckIscloseds= len(self.return_data)
#         print("The end points of each unclose polygon:")
#         print(points)
        return self.return_data, self.LcheckIscloseds
    
    
    def checkShortlength(self):
        dataList = self.customFetchAll(f"SELECT id, ST_Length(ST_Transform(geom2, {self.SRID})) as length from {self.tableName}")    
        # Initialize an empty list to store the tuples
        self.resLength = []
        # Iterate over each tuple in the list
        for item in dataList:
            # Check if the second value (index 1) is less than 1
            if item[1] < 10:
                # If it is, append the entire tuple to the result list
                self.resLength.append(item[0])
                
        self.LcheckShortlength = len(self.resLength)
#         print("List of Linestring that is below 1 meter")
#         print(self.resLength)
        return self.resLength, self.LcheckShortlength


    def checkShortArea(self):
        dataList = self.customFetchAll(f"SELECT id, ST_Area(ST_Transform(ST_GeomFromText(ST_AsText(geom), {self.SRID}), {self.SRID})) as area, asText(geom) from {self.tableName} where geotypes='Polygon'")    
#         print(dataList)
        # Initialize an empty list to store the tuples
        self.resArea = []
        # Iterate over each tuple in the list
        for item in dataList:
            # Check if the second value (index 1) is less than 1
#             print(item)
            if item[1] < 20:
#                 # If it is, append the entire tuple to the result list
                self.resArea.append(item[0])
        self.LcheckShortArea = len(self.resArea)
#         print("List of Polygon that is below 1 sqm")
#         print(self.resArea)
        return [self.resArea, self.LcheckShortArea]
    
    def checkCluster(self):
        cluster_data = self.customFetchAll(f"SELECT a.id AS point_id, b.id AS cluster_id, ST_Distance(a.geom, b.geom) AS distance FROM {self.tableName} AS a, {self.tableName} AS b WHERE a.id != b.id AND ST_Distance(a.geom, b.geom) < 1 AND ST_Distance(a.geom, b.geom) > 0.1")
        # Create a set to store unique tuples
        unique_tuples = set()

        # Iterate over the input data
        for tuple_value in cluster_data:
            # Check if the tuple meets the condition
            if (tuple_value[0], tuple_value[1], tuple_value[2]) not in unique_tuples and \
               (tuple_value[1], tuple_value[0], tuple_value[2]) not in unique_tuples:
                # Add the tuple to the set
                unique_tuples.add((tuple_value[0], tuple_value[1], tuple_value[2]))

        # Convert the set back to a list
        self.unique_data = list(unique_tuples)
        
#         print("Object in a cluster within 1m: ", self.unique_data)
        LcheckCluster = len(self.unique_data)
        return [self.unique_data, LcheckCluster]
    
    
    # Function to calculate the angle between three points
    def calculate_angle(self, p1, p2, p3):
        # Create vectors
        v1 = (p1[0] - p2[0], p1[1] - p2[1])
        v2 = (p3[0] - p2[0], p3[1] - p2[1])
        # Calculate dot product and magnitudes
        dot_product = np.dot(v1, v2)
        magnitude_v1 = np.linalg.norm(v1)
        magnitude_v2 = np.linalg.norm(v2)

        # Check if magnitudes are zero to avoid division by zero
        if magnitude_v1 == 0 or magnitude_v2 == 0:
            return np.nan  # Return NaN or some other value indicating undefined angle

        # Calculate the angle
        cos_angle = dot_product / (magnitude_v1 * magnitude_v2)
        # Avoid numerical issues due to floating-point arithmetic
        cos_angle = np.clip(cos_angle, -1.0, 1.0)
        angle = np.arccos(cos_angle)
        return np.degrees(angle)


    def parse_wkt(self, wkt_polygon):

        # Regular expression to extract coordinates from WKT polygon
        coord_pattern = re.compile(r'(\d+(\.\d+)?)\s+(\d+(\.\d+)?)')

        # Find all matches of the coordinate pattern in the WKT polygon
        matches = coord_pattern.findall(wkt_polygon)

        # Convert each match to a tuple of numerical values (x, y)
        points = [(float(x), float(y)) for x, _, y, _ in matches]

        return points

    def pseudo_node(self, geotypes):
        selected_data = self.customFetchAll(f"select id, AsText(geom) from {self.tableName} where geotypes='{geotypes}'")
        # Assuming you have a table "polygons" with a geometry column "geom"
        polygon_pseudo = []
        pseudo_nodes = []

        # Fetch the WKT representation of the polygon
        for wkt in selected_data: 
            # # Assuming you have a function to parse WKT and extract points
            # # This needs to be implemented based on your specific WKT format
            points = self.parse_wkt(wkt[1])  # Implement this function to extract points from WKT

            pseudo_node_count = 0
            # Adjust the angle threshold to detect angles significantly less than 180 degrees
            angle_threshold = 10  # Degrees

            # Initialize a list to store the indices of pseudo nodes
            
            

            # Calculate angles and identify pseudo nodes
            for i in range(len(points)):
                p1 = points[i - 1]  # Previous point
                p2 = points[i]      # Current point
                p3 = points[(i + 1) % len(points)]  # Next point, wrapping around using modulo

                angle = self.calculate_angle(p1, p2, p3)
                # Look for angles significantly less than 180 degrees
#                 print(angle)
                if angle < angle_threshold or angle == 180:
                    pseudo_nodes.append(i)
                    polygon_pseudo.append(wkt)

            # The count of pseudo nodes is the length of the pseudo_nodes list

#             print("Indices of pseudo nodes:", pseudo_nodes)
#             print("Number of pseudo nodes:", pseudo_node_count)
        
        self.Lpseudo_node = len(pseudo_nodes)
        return [self.Lpseudo_node, polygon_pseudo]

    
## Phase: Cleaning
    
    def removeShortArea(self,data):
        for x in data:
            cursor.execute(f"delete from {self.tableName} where id={x[0]}")
        conn.commit()
        print("The polygon that below 10sqm has been deleted")
        
    def removeShortLength(self,data):
        for x in data:
            cursor.execute(f"delete from {self.tableName} where id={x[0]}")
        conn.commit()
        print("The polygon that below 10sqm has been deleted")
    
    def splitPolygons(self,data):
        for r1_id, r2_id in data:
            split_data = self.customFetchOne(f"SELECT ST_AsText(ST_Intersection(r1.geom, r2.geom)) FROM {self.tableName} r1 CROSS JOIN {self.tableName} r2 WHERE r1.id = {r1_id} AND r2.id = {r2_id};")
            remain_data1 = self.customFetchOne(f"select ST_AsText(ST_Difference(r2.geom, r1.geom)) from {self.tableName} r1 cross join {self.tableName} r2 where r1.id = {r1_id} and r2.id = {r2_id};")
            remain_data2 = self.customFetchOne(f"select ST_AsText(ST_Difference(r1.geom, r2.geom)) from {self.tableName} r1 cross join {self.tableName} r2 where r1.id = {r1_id} and r2.id = {r2_id};")
    
#             insert into the table
            cursor.execute(f"INSERT INTO {self.tableName} (geotypes, geom) VALUES ('Polygon', ST_GeomFromText('{split_data[0]}', {self.SRID}));")
            cursor.execute(f"INSERT INTO {self.tableName} (geotypes, geom) VALUES ('Polygon', ST_GeomFromText('{remain_data1[0]}', {self.SRID}));")
            cursor.execute(f"INSERT INTO {self.tableName} (geotypes, geom) VALUES ('Polygon', ST_GeomFromText('{remain_data2[0]}', {self.SRID}));") 
        conn.commit()
        print("Polygon successfully splitted") 
        return self
        
    def sanitizeGeom(self, data):
        for identity in data:
            sanitize_data = self.customFetchOne(f"select ST_AsText(SanitizeGeometry(geom)) from {self.tableName} where id={identity}")
    #         new_geom = self.customFetchOne(f"INSERT INTO spatial_data (geotypes, geom) VALUES ('Polygon', ST_GeomFromText('{sanitize_data}',4326));")
            conn.commit()(f"UPDATE {self.tableName} SET geom=ST_GeomFromText('{sanitize_data}', {self.SRID}) WHERE id={identity}")
        
    def closingRing(self,data):
        for identity in data:
            closing_data = self.customFetchOne(f"select ST_AsText(EnsureClosedRings(geom)) from {self.tableName} where id={identity}")
    #         self.customFetchOne(f"INSERT INTO spatial_data (geotypes, geom) VALUES ('Polygon', ST_GeomFromText('{closing_data}',4326));")
            conn.commit()(f"UPDATE {self.tableName} SET geom=ST_GeomFromText('{closing_data}', {self.SRID}) WHERE id={identity}")
    
    def repeatPoints(self):
        df_id = self.customQuery(f"select id from {self.tableName}")
        l_id = df_id['id'].tolist()
        for item in l_id:
            repeated = self.customFetchOne(f"select ST_AsText(RemoveRepeatedPoints(geom)) from {self.tableName} where id={item}")
#             print(f"UPDATE {self.tableName} SET geom=ST_GeomFromText('{repeated}', {self.SRID}) WHERE id={item}")
            cursor.execute(f"UPDATE {self.tableName} SET geom=ST_GeomFromText('{repeated}', {self.SRID}) WHERE id={item}")
            conn.commit()
        print("all the duplicate point has been removed") 
        
    def parseMulti2Poly(self, Data):
        # Define the multipolygon string
        multipolygon_str = str(Data)
        # Parse the multipolygon string into a Shapely geometry object
        multipolygon = loads(multipolygon_str)
        # Get the first polygon from the multipolygon
        first_polygon = list(multipolygon.geoms)[0]  # Access the first geometry object within the MultiPolygon
        return first_polygon

    def selfIntersect(self):
        short_degenarate = []
        self_int = []
        all_poly = self.customFetchAll(f"select id, ST_AsText(EnsureClosedRings(geom)) as CloseGeom from {self.tableName} where geotypes='Polygon'")
        for x in all_poly:
            C_short_inter = self.customFetchOne(f"SELECT ST_AsText(ST_IsValidDetail(ST_GeomFromText('{x[1]}'))) as selfinter;")
            if C_short_inter is None:
                short_degenarate.append(x)
            else: 
                self_int.append(x)
        ## this section repair the self intersect polygon 
        for y in self_int:
            closing_data = self.customFetchOne(f"select ST_AsText(EnsureClosedRings(ST_GeomFromText('{y[1]}'))) as closeRing")
            makevalid_Test = self.customFetchOne(f"SELECT ST_AsText(ST_MakeValid(ST_GeomFromText('{closing_data}'))) AS split_result")
            one_poly = self.parseMulti2Poly(makevalid_Test)
            cursor.execute(f"UPDATE {self.tableName} SET geom=ST_GeomFromText('{one_poly}',{self.SRID}) WHERE id={y[0]}")
    
        ## this section repair the short degenerate
        for z in short_degenarate:
            sanitize_data = self.customFetchOne(f"select ST_AsText(SanitizeGeometry(ST_GeomFromText('{z[1]}'))) as sanitizeRing")
    #         update data
            cursor.execute(f"UPDATE {self.tableName} SET geom=ST_GeomFromText('{sanitize_data}', {self.SRID}) WHERE id={z[0]}")

        conn.commit()
        print("all the short degenerate and self intersect has been solved")
    
    def cleanPseudoNode(self, geotypes):
        selecteddata = self.customFetchAll(f"select id from {self.tableName} where geotypes= '{geotypes}'")
        for i in selecteddata:
            fix_data = self.customFetchOne(f"SELECT AsText(ST_SimplifyPreserveTopology(geom, 0.5)) FROM {self.tableName} where id={i[0]}")
            cursor.execute(f"update {self.tableName} set geom = ST_GeomFromText('{fix_data}',{self.SRID}) where id={i[0]};")
        conn.commit()
#             print(fix_data)
        print("all data which contain pseudo node has been fixed")


###     Phase checking result and delete
    def inspectTable(self, data):
        self.data = data
        print(self.data)
        
    def deleteTable(self):
        cursor.execute(f"DROP TABLE IF EXISTS {self.tableName}")
        conn.commit()
        print(f'{self.tableName} table successfully delete')
        return self
    
    def wkt2map(self, resultQuery):
#         data = self.customFetchAll(f"select id, geotypes, ST_AsText(geom) as geometry from {self.tableName} where geotypes='{geotypes}'")
        data = resultQuery
        # Fix WKT strings to ensure polygons are properly closed
        data = [(id, typ, wkt_string.rstrip(')') + ', {})'.format(wkt_string.split('((')[-1])) for id, typ, wkt_string in data]
        # Parse WKT strings and create Shapely geometry objects
        geometries = [wkt.loads(wkt_string) for _, _, wkt_string in data]
        # Create GeoDataFrame
        gdf = gpd.GeoDataFrame(data, columns=['ID', 'Type', 'WKT'], geometry=geometries, crs='EPSG:4326')
        # Create a Folium map centered at a specific location
        m = folium.Map(location=[15, 20], zoom_start=4)
        # Add GeoDataFrame to the map
        folium.GeoJson(gdf).add_to(m)
        # Display the map
        return m
    
    

In [258]:
def initiateBundle():
    BPNGeom_Poly = Geometry("BPN_Test",Example,"32748")
    BPNGeom_Poly = BPNGeom_Poly.initTable()
    
def convertBundle():
    BPNGeom_Poly = Geometry("BPN_Test",Example,"32748")
    BPNPoly = BPNGeom_Poly.convertPolygon()

def deleteBundle():
    BPNGeom_Poly = Geometry("BPN_Test",Example,"32748")
    BPNGeom_Poly.deleteTable()
    
def testBundle():
    BPNGeom_Poly = Geometry("BPN_Test",Example,"32748")
    ldup = BPNGeom_Poly.resDuplicate()
    lclose = BPNGeom_Poly.checkIsclosed()
    larea = BPNGeom_Poly.checkShortArea()
    lvalid = BPNGeom_Poly.checkIsValid("Polygon")
    lcross = BPNGeom_Poly.checkCrosses("Polygon")
    lpseudo = BPNGeom_Poly.pseudo_node('Polygon')
    lcluster = BPNGeom_Poly.checkCluster()
    llength = BPNGeom_Poly.checkShortlength()
    
#     print result
    print("Short Degenerate: ", llength[1])
    print("Short Entities: ", llength[1])
    print("Short segments: ", larea[1])
    print("Cross: ", lcross[2])
    print("Undershoot segment: ", lvalid[1])
    print("Undershoot vertex: ", lvalid[1])
    print("Duplicates: ", ldup[1])
    print("Cluster: ", lcluster[1])
    print("Pseudo nodes: ", lpseudo[0])
    print("Dangles: ", lclose[1])
    print("Silver Poligon: ", larea[1])
    
    
def cleanBundle():
    BPNGeom_Poly = Geometry("BPN_Test",Example,"32748")
#     larea = BPNGeom_Poly.checkShortArea()
#     llength = BPNGeom_Poly.checkShortlength()
    resultCrosses = BPNGeom_Poly.checkCrosses("Polygon")
    ## these part are for analysis implementation(cleaning)
    # to clean the LineString that has length below 1m
#     BPNGeom_Poly.removeShortLength(llength[0])
    
    # to remove duplicate point 
    BPNGeom_Poly.repeatPoints()
    
    # to clean pseudonode
    BPNGeom_Poly.cleanPseudoNode("Polygon")
    
    # to resolve short degenerate and self intersect 
    BPNGeom_Poly.selfIntersect()
    
    # to clean the polygon that has area below 1sqm
#     BPNGeom_Poly.removeShortArea(larea[0])
    
    # to split intersect polygon
    BPNGeom_Poly.splitPolygons(resultCrosses[1])

# Example = csv2coord("dataRekap.csv")
Example = csv2coord("braga3.csv")   

# Example = [
#     [[10, 10], [20,10], [20, 0], [11, 0], [11,11]]
# ]
    
# deleteBundle()
# initiateBundle()
# convertBundle()
# testBundle()
# cleanBundle()


# cursor.execute(f"INSERT INTO BPN_Test (geotypes, geom) VALUES ('Polygon', ST_GeomFromText('POLYGON((10 10, 20 10, 20 0, 10 0))',32857));")
# conn.commit()

BPNGeom_Poly = Geometry("BPN_Test",Example,"32748")
# BPNGeom_Poly.cleanPseudoNode("Polygon")

# print(customFetchAll(f"SELECT id, ST_Area(ST_Transform(ST_GeomFromText(ST_AsText(geom), 32748), 32748)) as area from BPN_Test where geotypes='Polygon'"))
# print(customFetchAll(f"SELECT ST_Area(ST_Transform(geom, 32748)) as area from BPN_Test"))

# data = BPNGeom_Poly.customFetchAll("select id from BPN_Test")
# print(data)

# data = BPNGeom_Poly.customFetchAll("select ST_AsText(geom) from BPN_Test")
# print(data)


# print(len(Example))



[(1,), (2,), (3,), (4,), (5,), (6,), (7,), (8,), (9,), (10,), (11,), (12,), (13,), (14,)]


In [259]:
# print(Example)
plotBefore(Example)

In [260]:
data5 = customFetchAll("SELECT id, geotypes, ST_AsText(geom) as geometry from BPN_Test")
# print(data5)
plotAfter(data5)


In [123]:
# instanciate the class and methods of geometry  Class

### first instanciate the geometry class by filling up the constructor
## the sequence is name of the table, coordinate, and the SRID 
# BPNGeom_line = Geometry("BPN",EgLinestring,"4326")
# BPNGeom_Poly = Geometry("BragaTest",EgPolygon2,"4326")
BPNGeom_Poly = Geometry("BPN_Test",Example,"32748")


## initiate the table 
# BPNGeom_line = BPNGeom_line.initTable()
# BPNGeom_Poly = BPNGeom_Poly.initTable()
 
## delete the current table 
# BPNGeom_line.deleteTable()
# BPNGeom_Poly.deleteTable()

## convert the coordinate into Point and insert the each line into the database
# BPNPoint = BPNGeom_line.convertPoint()

## convert the coordinate into linestring and insert the each line into the database
# BPNLine = BPNGeom_line.convertLinestring()

## convert the coordinate into Polygon and insert the each line into the database
# it only applied when all the coordinate inside the list is morethan 4 
# BPNPoly = BPNGeom_Poly.convertPolygon()

### Analysis part 

## these analysis are merely for linestring
## to check if there is any duplicate data point
# BPNGeom_line.checkDuplicate()

## to check the minimum length
# BPNGeom_line.checkShortlength()
## check if the linestring is not closed
# BPNGeom_line.checkIsclosed()
## to check if there is intersection between linestring 
# BPNGeom_line.checkCrosses("Linestring")

### these analysis are merely for Polygon
## to check if there is any duplicate data point
# BPNGeom_Poly.checkDuplicate()

## to check the mininmum area
# BPNGeom_Poly.checkShortArea()

## to check whether the polygon close properly or not (if short degenerate or intersect or over degenerate) will return the last point 
# BPNGeom_Poly.checkIsValid("Polygon")

## to check if there is intersection between polygon 
# resultCrosses = BPNGeom_Poly.checkCrosses("Polygon")
# resultCrosses[1]

## to check if there is pseudo node
# BPNGeom_Poly.pseudo_node('Polygon')

## to check if there is object in a cluster within 1 meter 
# BPNGeom_Poly.checkCluster()


### these part are for analysis implementation(cleaning)
## to split intersect polygon
# BPNGeom_Poly.splitPolygons(resultCrosses[1])

## to resolve short degenerate and self intersect 
# BPNGeom_Poly.selfIntersect()

## to remove duplicate point 
# BPNGeom_Poly.repeatPoints()

## to clean pseudonode
# BPNGeom_Poly.cleanPseudoNode("Polygon")

## to clean the polygon that has area below 1sqm
# BPNGeom_Poly.removeShortArea

## to clean the LineString that has length below 1m
# BPNGeom_line.removeShortLength


# # inspect.table and visualization
# BPNGeom_line.inspectTable(BPNGeom_line.customQuery(f"SELECT * from BPN"))


# BPNGeom_Poly.inspectTable(BPNGeom_Poly.customQuery(f"SELECT * from BPN"))

# data = BPNGeom_Poly.customFetchAll("select id, geotypes, ST_AsText(geom) as geometry from BPNUTM where geotypes='Polygon'")
# print(data)

# data2 = BPNGeom_Poly.customFetchAll("select id, ST_Area(ST_Transform(geom, 32748),32748) as geometry from BPNUTM ")
# print(data2)

# data3 = BPNGeom_Poly.customFetchAll("SELECT ST_Area(ST_Transform(ST_GeomFromText(ST_AsText(geom), 32748), 32748)) as area from BPNUTM")
# print(data3)


# BPNGeom_Poly.wkt2map(data)

# data =[(1, 2), (1, 5)]
# for r1_id, r2_id in data:
#     print(r1_id, r2_id)

# cursor.execute("INSERT INTO BPN (geotypes, geom) VALUES ('Polygon', ST_GeomFromText('POLYGON((0 10, 0 20, 10 20, 10 10, 0 10))', 4326));")
# conn.commit()

# data = BPNGeom_Poly.customFetchAll("select id from BPN_Test")
# print(data)

# print("")

# data4 = BPNGeom_Poly.customFetchAll("SELECT ST_Area(ST_Transform(geom, 32748)) as area from BPNUTM where id=1")
# print(data4)

# print("")

# data5 = BPNGeom_Poly.customFetchAll("SELECT ST_Area(ST_Transform(geom, 4326)) as area from BPNUTM where id=3")
# print(data5)

